# **Program Registrasi atau Training Classifier Pengenalan Citra Wajah**

### Arahkan colaboratory pada file project yang ingin dijalankan

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [23]:
%cd gdrive/My Drive/simple-face-recognition/
!ls

dataset		      FaceRecognition.ipynb	 output
dataset2	      FaceRecognition-v2.ipynb	 recognize_video.py
embeddings.txt	      images			 registrasi-training.ipynb
face_detection_model  openface_nn4.small2.v1.t7  registrasi-training-v2.ipynb


### Import libraries yang dibutuhkan

In [0]:
import numpy as np
import imutils
import pickle
import os
import cv2
from imutils import paths
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

### Inisialisasi variabel pendukung

In [0]:
dataset = 'dataset'
embeddings = 'output/embeddings.pickle'
recognizer_model = 'output/recognizer.pickle'
label_encoder = 'output/le.pickle'

### Inisialisasi Model Model yang Dibutuhkan

Model Pendeteksi Wajah

In [0]:
protoPath = 'face_detection_model/deploy.prototxt'
modelPath = 'face_detection_model/res10_300x300_ssd_iter_140000.caffemodel'
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

Model Pengekstraksi Fitur

In [0]:
embedding_model = 'openface_nn4.small2.v1.t7';
embedder = cv2.dnn.readNetFromTorch(embedding_model)

### Mengekstraksi fitur wajah oleh model deep learning

In [0]:
# mengambil path yang berisi dataset
imagePaths = list(paths.list_images('dataset'))

# inisialisasi fitur beserta labelnya
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

In [29]:
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,	len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]

	# load the image, resize it to have a width of 600 pixels (while maintaining the aspect ratio), and then grab the image dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300),	(104.0, 177.0, 123.0), swapRB=False, crop=False)

	# apply OpenCV's deep learning-based face detector to localize faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()

	# ensure at least one face was found
	if len(detections) > 0:
		# we're making the assumption that each image has only ONE face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		# ensure that the detection with the largest probability also means our minimum probability test (thus helping filter out weak detections)
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for the face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI and grab the ROI dimensions
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

			# ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue

			# construct a blob for the face ROI, then pass the blob through our face embedding model to obtain the 128-d quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,	(96, 96), (0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

			# add the name of the person + corresponding face embedding to their respective lists
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
			total += 1

[INFO] processing image 1/18
[INFO] processing image 2/18
[INFO] processing image 3/18
[INFO] processing image 4/18
[INFO] processing image 5/18
[INFO] processing image 6/18
[INFO] processing image 7/18
[INFO] processing image 8/18
[INFO] processing image 9/18
[INFO] processing image 10/18
[INFO] processing image 11/18
[INFO] processing image 12/18
[INFO] processing image 13/18
[INFO] processing image 14/18
[INFO] processing image 15/18
[INFO] processing image 16/18
[INFO] processing image 17/18
[INFO] processing image 18/18


### Training process

Mempersiapkan data yang akan ditraining

In [0]:
data = {"embeddings": knownEmbeddings, "names": knownNames}

Encode labels

In [31]:
le = LabelEncoder()
labels = le.fit_transform(data["names"])
le

LabelEncoder()

Train the model

In [32]:
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

Safe model recognition

In [0]:
f = open(recognizer_model, "wb")
f.write(pickle.dumps(recognizer))
f.close()

Safe name encoder

In [0]:
f = open(label_encoder, "wb")
f.write(pickle.dumps(le))
f.close()